In [1]:
# -*- coding: utf-8 -*from keras.models import Sequential
import numpy as np
from numpy.random import RandomState
import torch
import scipy.io as sio
import time
# from matlab import eng

In [2]:
#%% S11/S22/S12: 5000xN, S_all=[S11; S22; S12]
def ComputeVonMisesStress_all(S_all):

    S11_all=S_all[:, 0:5000]
    S22_all=S_all[:, 5000:10000]
    S12_all=S_all[:, 10000:15000]
    VM_all = S11_all**2 + S22_all**2 - S11_all*S22_all +3*S12_all**2
    VM_all = torch.sqrt(VM_all)

    return S11_all, S22_all, S12_all, VM_all


In [3]:
def ComputeError(A, B):
    MAE=np.zeros(A.shape[1])
    NMAE=np.zeros(A.shape[1])
    AE=np.zeros(A.shape[1])
    APE=np.zeros(A.shape[1])
    for n in range(0, A.shape[1]):
        a=A[:,n]
        b=B[:,n]
        c=torch.absolute(a-b)
        a_abs=torch.absolute(a)
        b_abs=torch.absolute(b)
        a_max=torch.max(a_abs)
        b_max=torch.max(b_abs)
        MAE[n]=torch.mean(c)
        NMAE[n]=MAE[n]/a_max
        AE[n]=torch.absolute(a_max-b_max)
        APE[n]=AE[n]/a_max
    return MAE, NMAE, AE, APE


In [4]:
class ErrorComputer():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, stress, pred_stress):
        self.stress = stress
        self.pred_stress = pred_stress
        self.MAE_S11 = []
        self.NMAE_S11 = []
        self.AE_S11 = []
        self.APE_S11 = []
        self.MAE_S22 = []
        self.NMAE_S22 = []
        self.AE_S22 = []
        self.APE_S22 =  []
        self.MAE_S12 = []
        self.NMAE_S12 = []
        self.AE_S12 = []
        self.APE_S12 = []
        self.MAE_VM = []
        self.NMAE_VM = []
        self.AE_VM = []
        self.APE_VM = []
                
    def compute(self):
        """Yield a batch of data after moving it to device"""
        [pred_S11, pred_S22, pred_S12, pred_VM] = ComputeVonMisesStress_all(self.pred_stress)
        [S11, S22, S12, VM] = ComputeVonMisesStress_all(self.stress)
        [self.MAE_S11, self.NMAE_S11, self.AE_S11, self.APE_S11] = ComputeError(pred_S11, S11)
        [self.MAE_S22, self.NMAE_S22, self.AE_S22, self.APE_S22] = ComputeError(pred_S22, S22)
        [self.MAE_S12, self.NMAE_S12, self.AE_S12, self.APE_S12] = ComputeError(pred_S12, S12)
        [self.MAE_VM, self.NMAE_VM, self.AE_VM, self.APE_VM] = ComputeError(pred_VM, VM)

    def print(self):
        print_MAE_mean = 'MAE mean\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_MAE_std = 'MAE std\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_NMAE_mean = 'NMAE mean\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_NMAE_std = 'NMAE std\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_AE_mean = 'AE mean\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_AE_std = 'AE std\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_APE_mean = 'APE mean\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print_APE_std = 'APE std\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}'
        print('Error\t\tS11\t\tS22\t\tS12\t\tVM')
        print('-------------------------------------------------------------------------')
        print(print_MAE_mean.format(np.mean(self.MAE_S11), np.mean(self.MAE_S22), np.mean(self.MAE_S12), np.mean(self.MAE_VM)))
        print(print_MAE_std.format(np.std(self.MAE_S11), np.std(self.MAE_S22), np.std(self.MAE_S12), np.std(self.MAE_VM)))
        print(print_NMAE_mean.format(np.mean(self.NMAE_S11), np.mean(self.NMAE_S22), np.mean(self.NMAE_S12), np.mean(self.NMAE_VM)))
        print(print_NMAE_std.format(np.std(self.NMAE_S11), np.std(self.NMAE_S22), np.std(self.NMAE_S12), np.std(self.NMAE_VM)))
        print(print_AE_mean.format(np.mean(self.AE_S11), np.mean(self.AE_S22), np.mean(self.AE_S12), np.mean(self.AE_VM)))
        print(print_AE_std.format(np.std(self.AE_S11), np.std(self.AE_S22), np.std(self.AE_S12), np.std(self.AE_VM)))
        print(print_APE_mean.format(np.mean(self.APE_S11), np.mean(self.APE_S22), np.mean(self.APE_S12), np.mean(self.APE_VM)))
        print(print_APE_std.format(np.std(self.APE_S11), np.std(self.APE_S22), np.std(self.APE_S12), np.std(self.APE_VM)))

In [5]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

In [6]:
class AllInOne(nn.Module):

    def __init__(self, shape_train, stress_train):
        super().__init__()
        self.shape_train = shape_train
        self.stress_train = stress_train[[0,1,3],:,:]
        self.loss_history = []

    def shapeModelWeight(self):
        # Finding weight 
        meanShape = torch.mean(self.shape_train, axis=0)
        X = (self.shape_train - meanShape)/656**(.5)
        [U, S, V] = torch.svd(X)
        proj = U[:,:3]/S[:3]
        return proj.t()

    def stress_shape(self, stress):
        out =torch.zeros(stress.shape[0],15000)
        for i in range(stress.shape[0]):
            temp11 = stress[i,0,:,:].reshape(5000)
            temp22 = stress[i,1,:,:].reshape(5000)
            temp12 = stress[i,2,:,:].reshape(5000)
            out[i,:] = torch.cat([temp11,temp22,temp12])
        return out

    def stressModelWeight(self):
        # Finding the weight of first convo-NN network
        temp11 = self.stress_train[0]
        temp22 = self.stress_train[1]
        temp12 = self.stress_train[2]
        Sdata1 = torch.cat([temp11.reshape(200,-1),temp22.reshape(200,-1),temp12.reshape(200,-1)], axis=0)
        covar_mat = Sdata1@Sdata1.t()/Sdata1.shape[0]
        [U1, S1, V1] = torch.svd(covar_mat)
        W1 = U1[:,:256].reshape(-1,3,10,20)

        # creating the first layer of stress encoding NN
        # and initializing weights
        stressEncodingModel_layer1 = nn.Conv2d(3, 256, kernel_size=[10,20], stride=[10,20])
        with torch.no_grad():
            stressEncodingModel_layer1.weight = Parameter(W1)
        # evaluating the output of first layer
        SData = torch.zeros(stress_train.shape[-1], 3, 50, 100)
        for i in range(self.stress_train.shape[-1]):
            SData[i,0,:,:] += temp11[:,i].reshape(50,100)
            SData[i,1,:,:] += temp22[:,i].reshape(50,100)
            SData[i,2,:,:] += temp12[:,i].reshape(50,100)
        output = stressEncodingModel_layer1(SData)
        self.SData = SData

        # Finding the weight of second convo-NN network
        Sdata2 = output.reshape(-1,output.shape[0])
        covar_mat = Sdata2@Sdata2.T/Sdata2.shape[0]
        [U2, S2, V2] = torch.svd(covar_mat)
        W2 = U2[:,:64].reshape(-1,256,5,5)

        return W1, W2

    def createModel(self):
        self.nonLinearModel = nn.Sequential(
            nn.Linear(3, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 64))
        
        W1, W2 = self.stressModelWeight()
        self.stressDecodingModel = nn.Sequential(
            nn.ConvTranspose2d(64, 256, kernel_size=[5,5], stride=1),
            nn.ConvTranspose2d(256, 3, kernel_size=[10,20], stride=[10,20]))
        with torch.no_grad():
            self.stressDecodingModel[0].weight = Parameter(W2)
            self.stressDecodingModel[1].weight = Parameter(W1)

        self.stressEncodingModel = nn.Sequential(
            nn.Conv2d(3, 256, kernel_size=[10,20], stride=[10,20]),
            nn.Conv2d(256, 64, kernel_size=[5,5], stride=1),
            nn.Flatten())
        with torch.no_grad():
            self.stressEncodingModel[0].weight = Parameter(W1)
            self.stressEncodingModel[1].weight = Parameter(W2)

        proj = self.shapeModelWeight()
        self.shapeEncodingModel = nn.Linear(15000,3)
        with torch.no_grad():
            self.shapeEncodingModel.weight = Parameter(proj)
        
    def encodingDecoding(self):
        # performing shape encoding and stress encoding
        self.encodedShape = self.shapeEncodingModel(self.shape_train.t())
        self.encodedStress = self.stressEncodingModel(self.SData)

    """# includes stress decoding
    def forward(self, xb):
        with torch.no_grad():
            self.stressDecodingModel[0].weight = Parameter(self.W1)
            self.stressDecodingModel[1].weight = Parameter(self.W2)
        print("performing decoding stress")
        t1 = time.perf_counter()
        output = self.stressDecodingModel(self.nonLinearModel(xb))
        # arraning the stress data 
        stress_S11 = output[:,0,:,:].reshape(5000,-1)
        stress_S22 = output[:,1,:,:].reshape(5000,-1)
        stress_S12 = output[:,2,:,:].reshape(5000,-1)
        decodedStress = torch.cat([stress_S11,stress_S22,stress_S12], axis=0)
        t2 = time.perf_counter()
        print('completed stress decoding. time taken = {}'.format(t2-t1))
        return decodedStress"""

    def training_step(self):
        pred_stress = self.nonLinearModel(self.encodedShape)                  
        loss = F.mse_loss(pred_stress, self.encodedStress) 
        return loss

    def closure(self):
        self.optimizer.zero_grad()
        loss = self.training_step()
        self.loss_history.append(loss)
        loss.backward(retain_graph=True)
        return loss

    def train(self, epochs, opt_func=torch.optim.Adam):
        self.optimizer = opt_func(self.nonLinearModel.parameters())

        for epoch in range(epochs):
            self.optimizer.step(self.closure)
            self.epoch_end(epoch, epochs, print_every=250)
    
    def epoch_end(self, epoch, epochs, print_every=100):
        if epoch == 0 or epoch == (epochs - 1) or epoch % print_every == 0 or print_every == 'all':
            print("Epoch [{}/{}], train_loss: {:.4f}".format(epoch, epochs, self.loss_history[epoch]))

    def validation_step(self):
        predStress_train = self.nonLinearModel(self.encodedShape)
        predStress_train = predStress_train[:,:,None,None]
        decodedStress_train = self.stressDecodingModel(predStress_train)
        temp = torch.zeros(self.stress_train.shape[-1],15000)
        for i  in range(self.stress_train.shape[-1]):
            temp[i,:] = self.stress_train[:,:,i].reshape(-1)
        trainError = ErrorComputer(temp, self.stress_shape(decodedStress_train))
        trainError.compute()
        trainError.print()

    def testing_step(self, shape_test, stress_test):
        encodedShape_test = self.shapeEncodingModel(shape_test.t())
        predStress_test = self.nonLinearModel(encodedShape_test)
        predStress_test = predStress_test[:,:,None,None]
        decodedStress_test = self.stressDecodingModel(predStress_test)
        temp = torch.zeros(stress_test.shape[-1],15000)
        for i  in range(stress_test.shape[-1]):
            temp[i,:] = stress_test[[0,1,3],:,i].reshape(-1)
        testError = ErrorComputer(temp, self.stress_shape(decodedStress_test))
        testError.compute()
        testError.print()



In [7]:
from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import random_split

In [8]:
ShapeDataFile = 'ShapeData.mat'
StressDataFile = 'StressData.mat'
MatData_shape=sio.loadmat(ShapeDataFile)
MatData_stress=sio.loadmat(StressDataFile)

In [9]:
stress_np = np.array(MatData_stress['StressData'])
shape_np = np.array(MatData_shape['ShapeData'])
# arraning the stress data for y
"""stress_S11 = stress_np[0,:,:]
stress_S22 = stress_np[1,:,:]
stress_S12 = stress_np[3,:,:]
stress_all = np.concatenate([stress_S11,stress_S22,stress_S12], axis=0)"""
# getting the tensor
shape_tensor = torch.from_numpy(shape_np)
shape_tensor = shape_tensor.float()
stress_tensor = torch.from_numpy(stress_np)
stress_tensor = stress_tensor.float()

In [10]:
# number of times training
n = 3
t_start = time.perf_counter()
for i in range(n):

    print("For epoch[{}/{}] :".format(i,n))
    t_e1 = time.perf_counter()

    # splitting training and testing set
    print("Splitting the dataset into training and testing data set")
    t1 = time.perf_counter()
    all_idx = np.arange(729)
    train_idx, test_idx  = random_split(all_idx, [656,73])
    shape_train = shape_tensor[:,train_idx]
    stress_train = stress_tensor[:,:,train_idx]
    shape_test = shape_tensor[:,test_idx]
    stress_test = stress_tensor[:,:,test_idx]
    t2 = time.perf_counter()
    print("Splitting is done. Time taken = {}".format(t2-t1))

    """creating the model and training the it"""
    # creating model
    model = AllInOne(shape_train, stress_train)
    # creating shaape and stress encoding model
    print("creating models for encoding, decoding and non-linear mapping:")
    t3 = time.perf_counter()
    model.createModel()
    t4 = time.perf_counter()
    print('Model created. time taken = {}'.format(t4-t3))
    # shape and stress encoding
    print("Encoding stress and shape : ")
    t5 = time.perf_counter()
    model.encodingDecoding()
    t6 = time.perf_counter()
    print('Shape and stress encoded. Time Taken = {}'.format(t6-t5))
    # training the model
    print("Training the model: ")
    t7 = time.perf_counter()
    model.train(5000)
    t8 = time.perf_counter()
    print('model trained. Time Taken = {}'.format(t8-t7))
    # performing validation step
    print("performing validation check on train:")
    t9 = time.perf_counter()
    model.validation_step()
    t10 = time.perf_counter()
    print('time taken in validation of final trained model with training data= {}'.format(t10-t9))
    # performing testing step
    print("performing validation check on train:")
    t11 = time.perf_counter()
    model.testing_step(shape_test, stress_test)
    t12 = time.perf_counter()
    print('time taken in validatoin of final trained model with testing data= {}'.format(t12-t11))

    t_e2 = time.perf_counter()
    print("Time taken for this epoch ={}".format(t_e2-t_e1))
    print("\n\n")

t_end = time.perf_counter()
print("Total Time taken ={}".format(t_end-t_start))

For epoch[0/3] :
Splitting the dataset into training and testing data set
Splitting is done. Time taken = 0.12905410000000117
creating models for encoding, decoding and non-linear mapping:
Model created. time taken = 73.51827890000001
Encoding stress and shape : 
Shape and stress encoded. Time Taken = 0.08262160000001018
Training the model: 
Epoch [0/5000], train_loss: 20390.3984
Epoch [250/5000], train_loss: 388.0349
Epoch [500/5000], train_loss: 291.1357
Epoch [750/5000], train_loss: 236.8018
Epoch [1000/5000], train_loss: 176.3658
Epoch [1250/5000], train_loss: 124.9627
Epoch [1500/5000], train_loss: 77.4766
Epoch [1750/5000], train_loss: 33.8256
Epoch [2000/5000], train_loss: 23.9821
Epoch [2250/5000], train_loss: 20.3511
Epoch [2500/5000], train_loss: 18.1838
Epoch [2750/5000], train_loss: 16.5063
Epoch [3000/5000], train_loss: 15.0738
Epoch [3250/5000], train_loss: 13.7909
Epoch [3500/5000], train_loss: 12.7354
Epoch [3750/5000], train_loss: 11.7962
Epoch [4000/5000], train_loss: